In [1]:
import cffi
import os
import numpy as np
import pandas as pd
from io import StringIO
from enum import Enum
from Bio import SeqIO
from Bio import Entrez
from pathlib import Path
from modules.auxiliary import DATA_SEQ_DIR
from modules.pytrsomix import TRScalculator, TRSanalyzer
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [2]:
# Prompt user for directory path
directory_path = input("Enter the directory path containing .fasta files: ")

# Get absolute path of the directory
directory_path = os.path.abspath(directory_path)

# Prompt user for tmin, tmax, and mode
tmin = int(input("Enter the value for tmin: "))
tmax = int(input("Enter the value for tmax: "))
mode = int(input("Enter the value for mode: "))

# Get list of .fasta files in the directory
fasta_files = [file for file in os.listdir(directory_path) if file.endswith(".fasta")]

# Initialize a list to store TRScalculator instances
trs_calculators = []

# Iterate over each .fasta file
for fasta_file in fasta_files:
    # Get absolute path of the fasta file
    fasta_path = os.path.join(directory_path, fasta_file).encode()
    
    # Define the file path for trs.txt (assuming it's located in the working directory)
    trs_file = os.path.abspath("trs.txt").encode()
    
    # Create TRScalculator instance and calculate TRS
    trs_calculator = TRScalculator(sequence=fasta_path, trs=trs_file, tmin=tmin, tmax=tmax, mode=mode)
    trs_calculator.calculate()
    
    # Append the TRScalculator instance to the list
    trs_calculators.append(trs_calculator)


Enter the directory path containing .fasta files: ./data/klebsiella
Enter the value for tmin: 50
Enter the value for tmax: 30000
Enter the value for mode: 1


In [26]:
# Initialize an empty list to store the results
results_list = []

# Iterate over each TRScalculator instance
for trs_calculator in trs_calculators:
    # Extract the result from the calculator
    result = trs_calculator.Result
    
    # Append the result to the list
    results_list.append(result)

# Concatenate all results into a single DataFrame
combined_results = pd.concat(results_list, ignore_index=True)

#Remove ">" from >SEQ column
combined_results[">SEQ"] = combined_results[">SEQ"].str.replace(">", "")

# Display the combined results
combined_results


,L-NoClass,L-No,LFS,Len(LFS),L-POS(LFS),R-POS(LFS),R-NoClass,R-No,RFS,Len(RFS),L-POS(RFS),R-POS(RFS),>SEQ,Len(SEQ),GENOME
0,2,4,CGGCGGCGG,9,2404,2412,2,4,CGGCGGCGG,9,3439,3447,CATTGGCCAGTCTCGTTTGACGATGCTGTTGCTGCAGCTTGACCAT...,1026,NC_016845.1
1,2,4,CGGCGGCGG,9,3439,3447,9,26,GCAGCAGCA,9,4374,4382,TATCATTTTCCGCCAGCACCGGATCGAGCTGACCGCTGAGCGTCAT...,926,NC_016845.1
2,9,26,GCAGCAGCA,9,4374,4382,10,29,CTGCTGCTG,9,4819,4827,GCGTCAGGGTGGTCCCCTGGATCTCCGCCGGCAGTTCATAGTGCGG...,436,NC_016845.1
3,10,29,CTGCTGCTG,9,4819,4827,10,28,GCTGCTGCT,9,6481,6489,CCATTGGGTAAATTCCTCGTCGCTCACCTGCAGCGCGCTGGGCACT...,1653,NC_016845.1
4,10,28,GCTGCTGCT,9,6481,6489,2,5,GGCGGCGGC,9,6778,6786,GAAGCACCCGGAAGCGATTAAAAACCCGTTCTTCCTGCTGGCGCCT...,288,NC_016845.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32107,4,12,TGGTGGTGG,9,5603843,5603851,14,41,TTCTTCTTC,9,5604969,5604977,TGCCGTCGACCATAAACAGCACCCGATCCGCCTGCTCAATCTCCTG...,1117,NZ_CP113789.1
32108,10,28,GCTGCTGCT,9,5605021,5605029,10,29,CTGCTGCTG,9,5606507,5606515,GAATAATGGTGACCAGGTTGCTGACGATGTAGTACACCACCAGGCC...,1477,NZ_CP113789.1
32109,10,29,CTGCTGCTG,9,5606507,5606515,3,8,CCACCACCA,9,5606927,5606935,GGGCTGCGGATTTTTGTCCTGCTCCCAGGCTTGCCAGATCATGAAA...,411,NZ_CP113789.1
32110,3,8,CCACCACCA,9,5606927,5606935,7,19,ACGACGACG,9,5607243,5607251,CGAAATCCATTGGCGGGAGTTCATGTTGACGCAAACGAAAACTTTC...,307,NZ_CP113789.1


In [1]:
# Extract directory name from the directory path
directory_name = os.path.basename(directory_path)

# Define the CSV file name
csv_file_name = directory_name + "_results.csv"

# Save combined_results to CSV file
combined_results.to_csv(csv_file_name, index=False)

print(f"Results saved to {csv_file_name}")

Enter the name of the file containing results: klebsiella_results.csv


NameError: name 'combined_results' is not defined

In [4]:
# Now we can load "folder_results.csv"
file_path = 
combined_results = pd.read_csv(file_path)
combined_results

NameError: name 'directory_name' is not defined

In [4]:
def extract_sequences(combined_results, l_chars, r_chars):
    # Create new columns SEQ_L and SEQ_R by slicing the sequence column
    combined_results['SEQ_L'] = combined_results['>SEQ'].str.slice(0, l_chars)
    combined_results['SEQ_R'] = combined_results['>SEQ'].str.slice(-r_chars)    
    return combined_results

l_chars = int(input("Enter the length of sequence to extract from the start: "))
r_chars = int(input("Enter the length of sequence to extract from the end: "))

combined_results = extract_sequences(combined_results, l_chars, r_chars)

Enter the length of sequence to extract from the start: 50
Enter the length of sequence to extract from the end: 50


In [29]:
combined_results

,L-NoClass,L-No,LFS,Len(LFS),L-POS(LFS),R-POS(LFS),R-NoClass,R-No,RFS,Len(RFS),L-POS(RFS),R-POS(RFS),>SEQ,Len(SEQ),GENOME,SEQ_L,SEQ_R
0,2,4,CGGCGGCGG,9,2404,2412,2,4,CGGCGGCGG,9,3439,3447,CATTGGCCAGTCTCGTTTGACGATGCTGTTGCTGCAGCTTGACCAT...,1026,NC_016845.1,CATTGGCCAGTCTCGTTTGACGATGCTGTTGCTGCAGCTTGACCAT...,CTGGTAGTCGCCTCGTTTGAGTTCGCCGGCGCTCATATCCCACAGG...
1,2,4,CGGCGGCGG,9,3439,3447,9,26,GCAGCAGCA,9,4374,4382,TATCATTTTCCGCCAGCACCGGATCGAGCTGACCGCTGAGCGTCAT...,926,NC_016845.1,TATCATTTTCCGCCAGCACCGGATCGAGCTGACCGCTGAGCGTCAT...,TTCAAAAGTAATGTGGATCACTTCCATATCGTGAAGTTTGAGCGGC...
2,9,26,GCAGCAGCA,9,4374,4382,10,29,CTGCTGCTG,9,4819,4827,GCGTCAGGGTGGTCCCCTGGATCTCCGCCGGCAGTTCATAGTGCGG...,436,NC_016845.1,GCGTCAGGGTGGTCCCCTGGATCTCCGCCGGCAGTTCATAGTGCGG...,AGCTGGAATATCAGCTCAAACACCGCGTCCGGCAGTTTAATATTGC...
3,10,29,CTGCTGCTG,9,4819,4827,10,28,GCTGCTGCT,9,6481,6489,CCATTGGGTAAATTCCTCGTCGCTCACCTGCAGCGCGCTGGGCACT...,1653,NC_016845.1,CCATTGGGTAAATTCCTCGTCGCTCACCTGCAGCGCGCTGGGCACT...,TTCTCCGTCGTGCTGCCCTCGTTGGTGCTGAACTATTTTGGCCAGG...
4,10,28,GCTGCTGCT,9,6481,6489,2,5,GGCGGCGGC,9,6778,6786,GAAGCACCCGGAAGCGATTAAAAACCCGTTCTTCCTGCTGGCGCCT...,288,NC_016845.1,GAAGCACCCGGAAGCGATTAAAAACCCGTTCTTCCTGCTGGCGCCT...,TACGTTTCTGTGGTCATCGTTATCGTCAGCTTTGAGCATTCGTCTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32107,4,12,TGGTGGTGG,9,5603843,5603851,14,41,TTCTTCTTC,9,5604969,5604977,TGCCGTCGACCATAAACAGCACCCGATCCGCCTGCTCAATCTCCTG...,1117,NZ_CP113789.1,TGCCGTCGACCATAAACAGCACCCGATCCGCCTGCTCAATCTCCTG...,ACTAATGAAGCCAACGCATCTGCGGCTTGAAGAATGAAGAGTATCA...
32108,10,28,GCTGCTGCT,9,5605021,5605029,10,29,CTGCTGCTG,9,5606507,5606515,GAATAATGGTGACCAGGTTGCTGACGATGTAGTACACCACCAGGCC...,1477,NZ_CP113789.1,GAATAATGGTGACCAGGTTGCTGACGATGTAGTACACCACCAGGCC...,ACGCCCTGGTCGGCGGCGCTACCCGCTGCTGTGGTCGTAGTCTGCG...
32109,10,29,CTGCTGCTG,9,5606507,5606515,3,8,CCACCACCA,9,5606927,5606935,GGGCTGCGGATTTTTGTCCTGCTCCCAGGCTTGCCAGATCATGAAA...,411,NZ_CP113789.1,GGGCTGCGGATTTTTGTCCTGCTCCCAGGCTTGCCAGATCATGAAA...,CAACGCTTCCGAGAGAGCACGGTTATCGAGGTCGGCAACCCCTCTT...
32110,3,8,CCACCACCA,9,5606927,5606935,7,19,ACGACGACG,9,5607243,5607251,CGAAATCCATTGGCGGGAGTTCATGTTGACGCAAACGAAAACTTTC...,307,NZ_CP113789.1,CGAAATCCATTGGCGGGAGTTCATGTTGACGCAAACGAAAACTTTC...,CAGGGTTAGCTTTATTACTTGGAAACGGTCAGACGAGCGCGGCCTT...


In [5]:
# Prompt the user to input their email address
user_email = input("Enter your email address: ")

# Set the user's email address (required by NCBI)
Entrez.email = user_email

# Function to fetch organism names from NCBI based on NCBI IDs
def fetch_organism_names(ncbi_ids):
    # Initialize a dictionary to store the organism names
    organism_map = {}
    # Iterate over the NCBI IDs
    for ncbi_id in ncbi_ids:
        # Fetch the record from NCBI
        handle = Entrez.efetch(db="nucleotide", id=ncbi_id, rettype="gb", retmode="text")
        record = handle.read()
        # Extract the organism information from the record
        for line in record.splitlines():
            if line.startswith("  ORGANISM"):
                organism = line.split("ORGANISM")[1].strip()
                # Replace spaces with underscores
                organism = organism.replace(" ", "_")
                organism_map[ncbi_id] = organism
                break  # Once organism information is found, break the loop
    return organism_map

# Read the DataFrame with the combined_results
ncbi_ids = combined_results["GENOME"].unique().tolist()

# Fetch organism names from NCBI
organism_map = fetch_organism_names(ncbi_ids)



Enter your email address: hsalamaga@ibb.waw.pl


In [6]:
# Map NCBI IDs to taxonomic names
combined_results['Taxonomic_Name'] = combined_results['GENOME'].map(organism_map)

# Identify unmatched genomes
unmatched_genomes = combined_results[combined_results['Taxonomic_Name'].isnull()]['GENOME'].unique()

if len(unmatched_genomes) > 0:
    print(f"Warning: Some genome IDs could not be matched with taxonomic names: {unmatched_genomes}")

# Extract sequences and create sequence IDs
combined_results['L_id'] = combined_results['Taxonomic_Name'] + '_L' + combined_results['L-No'].astype(str)
combined_results['R_id'] = combined_results['Taxonomic_Name'] + '_R' + combined_results['R-No'].astype(str)

# Select relevant columns
sequences_df = combined_results[['SEQ_L', 'SEQ_R', 'L_id', 'R_id']]

# Save left sequences to FASTA file
with open('sequences_L.fasta', 'w') as fasta_file:
    for _, row in sequences_df.iterrows():
        fasta_file.write(f'>{row["L_id"]}\n')
        fasta_file.write(f'{row["SEQ_L"]}\n')

# Save right sequences to FASTA file
with open('sequences_R.fasta', 'w') as fasta_file:
    for _, row in sequences_df.iterrows():
        fasta_file.write(f'>{row["R_id"]}\n')
        fasta_file.write(f'{row["SEQ_R"]}\n')


In [7]:
#There is possibility of several sequences having the same name as such we need to make sure that all ids are unique
def rename_sequences(input_file, output_file):
    """
    Renames sequences in a FASTA file.

    Parameters:
        input_file (str): Path to the input FASTA file.
        output_file (str): Path to the output FASTA file with renamed sequences.
    """
    # Open input and output FASTA files
    with open(input_file, "r") as input_handle, open(output_file, "w") as output_handle:
        # Read sequences from input FASTA file and write to output with new names
        for i, record in enumerate(SeqIO.parse(input_handle, "fasta"), start=1):
            # Extract the original ID
            original_id = record.id
            # Rename the sequence using a combination of original ID and sequence number
            new_id = f"{original_id}_{i}"
            record.id = new_id
            record.description = ""
            # Write the updated record to the output file
            SeqIO.write(record, output_handle, "fasta")

# Call the function to rename sequences
rename_sequences("sequences_R.fasta", "sequences_R_unique.fasta")
rename_sequences("sequences_L.fasta","sequences_L_unique.fasta")
# This function incorporates index into sequence name making it easy to find the original sequence
# Remove original .fasta files
os.remove("sequences_R.fasta")
os.remove("sequences_L.fasta")

In [8]:
import subprocess
def run_cdhit(cdhit_path, input_file, output_file, c=0.9, n=5, d=0, m="t",g=0,G=1,sc=0):
    """
    Run CD-HIT program from Python.

    Parameters:
        cdhit_path (str): Path to the CD-HIT executable.
        input_file (str): Path to the input FASTA file.
        output_file (str): Path to the output file.
        c (float): Sequence identity threshold. Default is 0.9.
        n (int): Word length, default is 5.
        d (int): Bandwidth of alignment, default is 0.
        m (str): Memory limit, default is "t" for unlimited.
        g (int): def 0 
        G (int): def 0
        sc (int): def 0
    """
    # Construct the CD-HIT command
    cmd = [cdhit_path, "-i", input_file, "-o", output_file, "-c", str(c), "-n", str(n), "-d", str(d), "-M", m,
          "-g",str(g), "-G", str(G),"-sc",str(sc)]

    # Run CD-HIT command as a subprocess
    subprocess.run(cmd)

# Example usage:
run_cdhit("/home/hubert/cdhit-master/cd-hit-est","sequences_L_unique.fasta","sequences_L_unique_cdhit",
          c=1, n=5, d=0, m="t",g=1,sc=1)
run_cdhit("/home/hubert/cdhit-master/cd-hit-est","sequences_R_unique.fasta","sequences_R_unique_cdhit",
          c=1, n=5, d=0, m="t",g=1,sc=1)
os.remove("sequences_L_unique_cdhit")
os.remove("sequences_R_unique_cdhit")

In [9]:
def extract_sequence_ids(input_file, output_file):
    """
    Extract sequence IDs from clusters with more than 2 sequences and save them to a file.

    Parameters:
        input_file (str): Path to the input cluster data file.
        output_file (str): Path to the output file to save sequence IDs.

    Returns:
        None
    """
    with open(input_file, "r") as f, open(output_file, "w") as outfile:
        current_cluster = []
        for line in f:
            if line.startswith(">Cluster"):
                if len(current_cluster) >= 2:
                    for seq in current_cluster:
                        outfile.write(seq)
                current_cluster = []  # Clear current cluster
            elif line.strip():  # Proceed if the line is not empty
                current_cluster.append(line)  # Add sequence to current cluster
        if len(current_cluster) >= 2:
            for seq in current_cluster:
                outfile.write(seq)
# Example usage
input_file = "sequences_L_unique_cdhit.clstr"
output_file = "L_clusters.txt"
extract_sequence_ids(input_file, output_file)

input_file = "sequences_R_unique_cdhit.clstr"
output_file ="R_clusters.txt"
extract_sequence_ids(input_file,output_file)


def clean_sequence_ids(input_file):
    """
    Clean sequence IDs file by removing prefixes and suffixes.

    Parameters:
        input_file (str): Path to the input file with sequence IDs.

    Returns:
        None
    """
    with open(input_file, "r") as infile:
        lines = infile.readlines()

    with open(input_file, "w") as outfile:
        for line in lines:
            sequence_id = line.split(">")[1].split("...")[0].strip()
            outfile.write(">" + sequence_id + "\n")

# Example usage
input_file = "L_clusters.txt"
clean_sequence_ids(input_file)

input_file = "R_clusters.txt"
clean_sequence_ids(input_file)

In [10]:
"""
Remove entries in L/R_clusters from the .fasta file 
"""
def read_fasta_ids(filename):
    fasta_ids = set()
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('>'):
                fasta_id = line.strip()[1:]
                fasta_ids.add(fasta_id)
    return fasta_ids

def filter_fasta_file(input_file, output_file, fasta_ids_to_remove):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        remove_entry = False
        for line in f_in:
            if line.startswith('>'):
                current_fasta_id = line.strip()[1:]
                if current_fasta_id in fasta_ids_to_remove:
                    remove_entry = True
                else:
                    remove_entry = False
            if not remove_entry:
                f_out.write(line)

# Read fasta IDs from the L_clusters file
fasta_ids_to_remove = read_fasta_ids("L_clusters.txt")

# Filter the sequences_L_unique.fasta file
filter_fasta_file("sequences_L_unique.fasta", "filtered_sequences_L_unique.fasta", fasta_ids_to_remove)

print("Filtered file created successfully!")

fasta_ids_to_remove = read_fasta_ids("R_clusters.txt")

filter_fasta_file("sequences_R_unique.fasta", "filtered_sequences_R_unique.fasta", fasta_ids_to_remove)

Filtered file created successfully!


In [11]:
"""
Extract only ids matching those in L/R Cluster files they might come in handy 
and will be helpful to verify whether or not our way of thinking is valid
"""
def filter_fasta_file_clusters(input_file, output_file, fasta_ids_to_include):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        include_entry = False
        for line in f_in:
            if line.startswith('>'):
                current_fasta_id = line.strip()[1:]
                if current_fasta_id in fasta_ids_to_include:
                    include_entry = True
                    f_out.write(line)
                else:
                    include_entry = False
            elif include_entry:
                f_out.write(line)

# Read fasta IDs from the L_clusters file
fasta_ids_to_include_L = read_fasta_ids("L_clusters.txt")

# Filter the sequences_L_unique.fasta file
filter_fasta_file_clusters("sequences_L_unique.fasta", "sequences_L_in_clusters.fasta", fasta_ids_to_include_L)

print("Left cluster sequences file created successfully!")

# Read fasta IDs from the R_clusters file
fasta_ids_to_include_R = read_fasta_ids("R_clusters.txt")

# Filter the sequences_R_unique.fasta file
filter_fasta_file_clusters("sequences_R_unique.fasta", "sequences_R_in_clusters.fasta", fasta_ids_to_include_R)

print("Right cluster sequences file created successfully!")
#for some reason the order of sequences is different but it works nonetheless

Left cluster sequences file created successfully!
Right cluster sequences file created successfully!
